# Build and imports

In [ ]:
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip
!pip3 install netCDF4

In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt


# def indices

In [ ]:

def heat_index(R,TC):
  #input output both in celsius
  T=1.8*TC+32

  c1=-42.379
  c2=2.04901523
  c3=10.14333127
  c4=-0.22475541
  c5=-6.83783*0.001
  c6=-5.481717*0.01
  c7=1.22874*0.001
  c8=8.5282*0.0001
  c9=-1.99*0.000001
  
  HI=c1+c2*T+c3*R+c4*T*R+c5*T*T+c6*R*R+c7*T*T*R+c8*T*R*R+c9*T*T*R*R
  HIC=(HI-32)/1.8
  return HIC


def thresh_heat_index(ts_rh,ts_tas):
  thresh=32
  ts_tas[ts_rh<40]=27
  ts_rh[ts_rh<40]=40
  ts_tas[ts_tas<27]=27
  ts_rh[ts_tas<27]=40
  

 # print(ts_rh.shape,ts_tas.shape)
  daily_ts=heat_index(ts_rh,ts_tas)

  return np.sum([daily_ts>=thresh])

def heat_index_annual(daily_rh,daily_tas,st,en):
  heat_index_annual=np.zeros((en-st+1))
  days_iter=0
  for yeaR in range(en-st+1):
    daily_year_rh=daily_rh[days_iter:days_iter+uu.isleapyear(st)+365]
    daily_year_tas=daily_tas[days_iter:days_iter+uu.isleapyear(st)+365]
    heat_index_annual[yeaR]=thresh_heat_index(daily_year_rh,daily_year_tas)
    days_iter=days_iter+uu.isleapyear(st)+365
  return heat_index_annual



# defs

In [ ]:
from numpy import meshgrid
from mpl_toolkits.basemap import Basemap
def map_plot_cyl(data_lat_x_lon,lat_cen,lon_cen,map_bounds,clab,cmap,y_limit,ax,mp_spacing=10,subplot_iter=1,loff=1,ex='both'):

  lat_box=uu.cen_to_box(lat_cen)
  lon_box=uu.cen_to_box(lon_cen)

  map = Basemap(projection='cyl',resolution='l',
                llcrnrlon=map_bounds[0], 
                urcrnrlat=map_bounds[1],
                urcrnrlon=map_bounds[2], 
                llcrnrlat=map_bounds[3])


  xx, yy = meshgrid(lon_box,lat_box )
  #return  dimenion of lat * lon
  map.pcolor(xx, yy, data_lat_x_lon,cmap=cmap)
  map.drawmeridians(np.arange(-180,180,mp_spacing),labels=[0,0,0,1], linewidth=1.0) #longitudes
  if loff==0:
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[1,0,0,0], linewidth=1.0) #latitudes
  else:
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[0,0,0,0], linewidth=1.0) #latitudes
  cbar = map.colorbar(extend=ex)
  #cbar.ax.set_yticklabels(['0','1','2','>3'])
  cbar.set_label(clab, rotation=90)
  #map.drawcountries(linewidth=1)
  map.readshapefile('/content/drive/MyDrive/Shapefiles/ind', 'IND', linewidth=1.0)

  map.drawcoastlines(linewidth=1)
  lxx=0.05
  lyy=0.92
  lcol='k'
  lsize=24
  lfamily='Tahoma'
  lweight=800
 # y_limit=[0,30]
  #clab='days'
  
  plt.clim(y_limit)
  plt.text(lxx, lyy, chr(subplot_iter+96), horizontalalignment='center',verticalalignment='center', transform=ax.transAxes,fontweight=lweight,
           fontfamily=lfamily,fontsize=lsize,color=lcol)


# Import data

## rh and tas

In [ ]:
map_bounds=[65, 40, 100, 5]

gcm_names=['MIROC_ESM','CanESM2','HadGEM2_ES','GFDL_CM3']#

rh_obs=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_obs_pred.npy')
tas_obs=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_obs_pred.npy')


i=0
gcm_name=gcm_names[i]
rh_gcm_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_gcm_'+gcm_name+'_pred.npy')
rh_bcsd_gcm_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+gcm_name+'.npy')

tas_gcm_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_gcm_'+gcm_name+'_pred.npy')
tas_bcsd_gcm_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+gcm_name+'.npy')

tas_gcm_meta_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_metadata_'+gcm_name+'.npy',allow_pickle=True)

st=tas_gcm_meta_01.item()['st']
en=tas_gcm_meta_01.item()['en']
yr_split=tas_gcm_meta_01.item()['yr_split']

lat_obs=tas_gcm_meta_01.item()['lat_obs']
lon_obs=tas_gcm_meta_01.item()['lon_obs']

lat_gcm_01=tas_gcm_meta_01.item()['lat_gcm']
lon_gcm_01=tas_gcm_meta_01.item()['lon_gcm']



i=1
gcm_name=gcm_names[i]
rh_gcm_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_gcm_'+gcm_name+'_pred.npy')
rh_bcsd_gcm_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+gcm_name+'.npy')

tas_gcm_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_gcm_'+gcm_name+'_pred.npy')
tas_bcsd_gcm_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+gcm_name+'.npy')

tas_gcm_meta_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_metadata_'+gcm_name+'.npy',allow_pickle=True)

lat_gcm_02=tas_gcm_meta_02.item()['lat_gcm']
lon_gcm_02=tas_gcm_meta_02.item()['lon_gcm']


i=2
gcm_name=gcm_names[i]
rh_gcm_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_gcm_'+gcm_name+'_pred.npy')
rh_bcsd_gcm_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+gcm_name+'.npy')

tas_gcm_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_gcm_'+gcm_name+'_pred.npy')
tas_bcsd_gcm_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+gcm_name+'.npy')

tas_gcm_meta_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_metadata_'+gcm_name+'.npy',allow_pickle=True)

lat_gcm_03=tas_gcm_meta_03.item()['lat_gcm']
lon_gcm_03=tas_gcm_meta_03.item()['lon_gcm']


i=3
gcm_name=gcm_names[i]
rh_gcm_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_gcm_'+gcm_name+'_pred.npy')
rh_bcsd_gcm_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+gcm_name+'.npy')

tas_gcm_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_gcm_'+gcm_name+'_pred.npy')
tas_bcsd_gcm_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+gcm_name+'.npy')

tas_gcm_meta_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_metadata_'+gcm_name+'.npy',allow_pickle=True)

lat_gcm_04=tas_gcm_meta_04.item()['lat_gcm']
lon_gcm_04=tas_gcm_meta_04.item()['lon_gcm']



In [ ]:

j=0
out_id='rnd_'+str(j)
rh_bcsd_rnd_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+out_id+'.npy')
tas_bcsd_rnd_01=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+out_id+'.npy')

j=1
out_id='rnd_'+str(j)
rh_bcsd_rnd_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+out_id+'.npy')
tas_bcsd_rnd_02=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+out_id+'.npy')

j=2
out_id='rnd_'+str(j)
rh_bcsd_rnd_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+out_id+'.npy')
tas_bcsd_rnd_03=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+out_id+'.npy')


j=3
out_id='rnd_'+str(j)
rh_bcsd_rnd_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/rh_BCSD_'+out_id+'.npy')
tas_bcsd_rnd_04=np.load('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/tas_BCSD_'+out_id+'.npy')



## creates mask

In [ ]:
!pip install geopandas

In [ ]:

import geopandas as gpd
from shapely.geometry  import Point
from shapely.geometry import shape

import numpy.ma as ma
from numpy import meshgrid

fo=gpd.read_file("/content/drive/MyDrive/Shapefiles/ind.shp")
fo.plot()
shp_geom = shape(fo.geometry[0]).buffer(1)

xxc, yyc = meshgrid(lon_obs.data,lat_obs.data)
mask_shape=np.empty_like(xxc)

N1=xxc.shape[0]
N2=xxc.shape[1]
for lat_i in range(N1):
  for lon_i in range(N2):
    xi=xxc[lat_i,lon_i]
    yi=yyc[lat_i,lon_i]
    m1= (shp_geom.contains(Point(xi, yi)) or shp_geom.touches(Point(xi, yi)))

    mask_shape[lat_i,lon_i]=not m1

## fined gcm

In [ ]:
rh_fined_gcm_01=np.zeros(rh_obs.shape)
rh_fined_gcm_02=np.zeros(rh_obs.shape)
rh_fined_gcm_03=np.zeros(rh_obs.shape)
rh_fined_gcm_04=np.zeros(rh_obs.shape)

tas_fined_gcm_01=np.zeros(tas_obs.shape)
tas_fined_gcm_02=np.zeros(tas_obs.shape)
tas_fined_gcm_03=np.zeros(tas_obs.shape)
tas_fined_gcm_04=np.zeros(tas_obs.shape)

for t in range(tas_obs.shape[0]):
  rh_fined_gcm_01[t,:,:]=uu.regrid(rh_gcm_01[t,:,:],lat_gcm_01,lon_gcm_01,lat_obs,lon_obs)
  rh_fined_gcm_02[t,:,:]=uu.regrid(rh_gcm_02[t,:,:],lat_gcm_02,lon_gcm_02,lat_obs,lon_obs)
  rh_fined_gcm_03[t,:,:]=uu.regrid(rh_gcm_03[t,:,:],lat_gcm_03,lon_gcm_03,lat_obs,lon_obs)
  rh_fined_gcm_04[t,:,:]=uu.regrid(rh_gcm_04[t,:,:],lat_gcm_04,lon_gcm_04,lat_obs,lon_obs)
  
  tas_fined_gcm_01[t,:,:]=uu.regrid(tas_gcm_01[t,:,:],lat_gcm_01,lon_gcm_01,lat_obs,lon_obs)
  tas_fined_gcm_02[t,:,:]=uu.regrid(tas_gcm_02[t,:,:],lat_gcm_02,lon_gcm_02,lat_obs,lon_obs)
  tas_fined_gcm_03[t,:,:]=uu.regrid(tas_gcm_03[t,:,:],lat_gcm_03,lon_gcm_03,lat_obs,lon_obs)
  tas_fined_gcm_04[t,:,:]=uu.regrid(tas_gcm_04[t,:,:],lat_gcm_04,lon_gcm_04,lat_obs,lon_obs)



In [ ]:
data_rh=[rh_obs,
      rh_fined_gcm_01,
      rh_fined_gcm_02,
      rh_fined_gcm_03,
      rh_fined_gcm_04,
      rh_bcsd_gcm_01,
      rh_bcsd_gcm_02,
      rh_bcsd_gcm_03,
      rh_bcsd_gcm_04,
      rh_bcsd_rnd_01,
      rh_bcsd_rnd_02,
      rh_bcsd_rnd_03,
      rh_bcsd_rnd_04
      ]

data_tas=[tas_obs,
      tas_fined_gcm_01,
      tas_fined_gcm_02,
      tas_fined_gcm_03,
      tas_fined_gcm_04,
      tas_bcsd_gcm_01,
      tas_bcsd_gcm_02,
      tas_bcsd_gcm_03,
      tas_bcsd_gcm_04,
      tas_bcsd_rnd_01,
      tas_bcsd_rnd_02,
      tas_bcsd_rnd_03,
      tas_bcsd_rnd_04
      ]


plt.figure(figsize=(10,10))
for i in range(13):
  plt.subplot(4,4,i+1)
  plt.pcolor(data_rh[i][250,:,:])
  plt.colorbar()

plt.figure(figsize=(20,10))
for i in range(13):
  plt.subplot(4,4,i+1)
  plt.plot(data_rh[i][:,7,7])


plt.figure(figsize=(10,10))
for i in range(13):
  plt.subplot(4,4,i+1)
  plt.pcolor(data_tas[i][250,:,:])
  plt.colorbar()

plt.figure(figsize=(20,10))
for i in range(13):
  plt.subplot(4,4,i+1)
  plt.plot(data_tas[i][:,7,7])



# Get spatial mean

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
import pandas as pd 
from shapely.geometry  import Point
from shapely.geometry import shape
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
  
# creating the dataframe

titles=['Obs',
        'GCM-M',
        'GCM-C',
        'GCM-H',
        'GCM-G',
        'BC-M',
        'BC-C',
        'BC-H',
        'BC-G',
        'BC-R0',
        'BC-R1',
        'BC-R2',
        'BC-R3']

In [ ]:
def get_regional_rmse(data_0,data_1,lat_obs_cru,lon_obs_cru,region_shapes):
  XX, YY = meshgrid(lon_obs_cru,lat_obs_cru)
  N1=XX.shape[0]
  N2=XX.shape[1]

  data_out=[]

  for shp in range(len(region_shapes.geometry)):
    data_temp=[]
    shp_geom = shape(region_shapes.geometry[shp])

    for lat_i in range(N1):
      for lon_i in range(N2):
        xi=XX[lat_i,lon_i]
        yi=YY[lat_i,lon_i]
        m1= (shp_geom.contains(Point(xi, yi)) or shp_geom.touches(Point(xi, yi)))
        if m1:
          data_temp.append((data_1[lat_i,lon_i]-data_0[lat_i,lon_i])**2)
    
    data_out.append(np.sqrt(np.nanmean(np.array(data_temp))))
  return data_out

def get_regional_corr(data_0,data_1,lat_obs_cru,lon_obs_cru,region_shapes):
  XX, YY = meshgrid(lon_obs_cru,lat_obs_cru)
  N1=XX.shape[0]
  N2=XX.shape[1]

  data_out=[]

  for shp in range(len(region_shapes.geometry)):
    data_temp_0=[]
    data_temp_1=[]
    shp_geom = shape(region_shapes.geometry[shp])

    for lat_i in range(N1):
      for lon_i in range(N2):
        xi=XX[lat_i,lon_i]
        yi=YY[lat_i,lon_i]
        m1= (shp_geom.contains(Point(xi, yi)) or shp_geom.touches(Point(xi, yi)))
        if m1:
          data_temp_0.append(data_0[lat_i,lon_i])
          data_temp_1.append(data_1[lat_i,lon_i])
    
    data_out.append(np.corrcoef(data_temp_0,data_temp_1)[0,1])
  return data_out

In [ ]:
region_shapes = gpd.read_file("/content/drive/MyDrive/Shapefiles/zones_imd.shp")
S= len(region_shapes['State_Name'])
print(S)

In [ ]:

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

plt.figure(figsize=(10,10))
ax=plt.subplot(1,1,1)

buu = cm.get_cmap('Blues', 128)
newcmp = ListedColormap(buu(np.linspace(0.25, 0.75, 10)), name='OrangeBlue')

region_shapes.plot(column='State_Name', ax=ax,cmap=newcmp,legend=True,linewidth=2.0, edgecolor='0.9',linestyle="-")
plt.xlabel('Longitudes')
plt.ylabel('Latitudes')

In [ ]:


def get_corr_n_rmse(indic,lat_obs,lon_obs):
  S=len(region_shapes.geometry)
  NN=len(indic)
  #data_corr=np.zeros((S,13,13))
  data_rmse=np.zeros((S,NN,NN))*np.nan
  for i in range(NN):
    for j in range(i+1):
      rmse_1=get_regional_rmse(indic[i],indic[j],lat_obs,lon_obs,region_shapes)
      #corr_1=get_regional_corr(indic[i],indic[j],lat_obs,lon_obs,region_shapes)
      #data_corr[:,i,j]=np.array(corr_1)
      data_rmse[:,i,j]=np.array(rmse_1)

  return data_rmse#data_corr,



In [ ]:
len([1,2,3][0:1])

In [ ]:


def plot_heatmap(data_corr,ylim,titles,fmt,cmap,title1):
  plt.figure(figsize=(25,13.75))
  S=data_corr.shape[0]
  for subs in range(S):
    plt.subplot(2,3,subs+1)
    
    data_set = data_corr[subs,:,:]
    sns.heatmap(data_set , linewidth = 2 , cmap = cmap, 
                annot=True,
                fmt=fmt ,square=True,
                xticklabels=titles,
                yticklabels=titles,vmin=ylim[0],vmax=ylim[1])
    plt.title(region_shapes['State_Name'][subs])
  plt.savefig('/content/drive/MyDrive/Figures/Figure3/indices/imd_rmse_indices_'+title1+'.png')
  plt.show()

In [ ]:
titles=['Obs',
        'GCM-M',
        'GCM-C',
        'GCM-H',
        'GCM-G',
        'BC-M',
        'BC-C',
        'BC-H',
        'BC-G',
        'BC-R0',
        'BC-R1',
        'BC-R2',
        'BC-R3']

# Compute indices

## def general function

In [ ]:
titles123=['Obs',
        'MIROC_ESM',
        'CanESM2',
        'HadGEM_AO',
        'GFDL_CM3',
        'BCSD-MIROC_ESM',
        'BCSD-CanESM2',
        'BCSD-HadGEM2_ES',
        'BCSD-GFDL_CM3',
        'BCSD-Rnd_0',
        'BCSD-Rnd_1',
        'BCSD-Rnd_2',
        'BCSD-Rnd_3'
        ]
titles=['Obs',
        'GCM-M',
        'GCM-C',
        'GCM-H',
        'GCM-G',
        'BC-M',
        'BC-C',
        'BC-H',
        'BC-G',
        'BC-R0',
        'BC-R1',
        'BC-R2',
        'BC-R3'
        ]

lofff=[0,0,1,1,1,0,1,1,1,0,1,1,1]

import numpy.ma as ma
def spatial_ind(data_rh,data_tas):

  D=13
  R20_all=np.zeros((D,N1,N2))
  for d in range(D):
    for i in range(N1):
      for j in range(N2):
        R20_all[d,i,j]=np.mean(heat_index_annual(data_rh[d][:,i,j],data_tas[d][:,i,j],yr_split,en))
    print(str((d+1)/D*100))
  return R20_all

def plot_indices(R20_all,titles,title1,clab,y_limit,ex='both',cmap='jet'):
  
  fig=plt.figure(figsize=(24,24))
  D=13

  for d in range(D):
    ax=plt.subplot(4,4,d+4)
    map_plot_cyl(ma.masked_array(R20_all[d,:,:], mask=mask_shape),lat_obs,lon_obs,map_bounds,clab,cmap,y_limit,ax,mp_spacing=10,subplot_iter=d+1,loff=lofff[d],ex=ex)
    plt.title(titles[d])
    if d == 3:
      plt.text(0.5, 1.15, title1, horizontalalignment='center',verticalalignment='center', transform=ax.transAxes,fontweight=600,
              fontfamily='Tahoma',fontsize=20,color='k')
  plt.savefig('/content/drive/MyDrive/Figures/Figure3/indices/imd_indices_'+title1+'.png')
  #plt.savefig('/content/drive/MyDrive/BiasCorr/plots/imd_indices_'+title1+'.png')


## heat index

In [ ]:
hit2=spatial_ind(data_rh,data_tas)
titles123=['Obs',
        'MIROC_ESM',
        'CanESM2',
        'HadGEM2_ES',
        'GFDL_CM3',
        'BCSD-MIROC_ESM',
        'BCSD-CanESM2',
        'BCSD-HadGEM2_ES',
        'BCSD-GFDL_CM3',
        'BCSD-Rnd_0',
        'BCSD-Rnd_1',
        'BCSD-Rnd_2',
        'BCSD-Rnd_3']
plot_indices(hit2,titles123,'HI32','# of days',[0,100],ex='max',cmap='YlOrRd')

#R10mma,R10mmb=get_corr_n_rmse(R10mmindic)
#plot_heatmap(R10mma,[-1,1],titles,".1f",'RdBu_r')
#plot_heatmap(R10mmb,[0,100],titles,".0f",'Reds')

In [ ]:
b=get_corr_n_rmse(hit2,lat_obs,lon_obs)
plot_heatmap(b,[0,100],titles,".0f",'Reds','HI32')

In [ ]:
np.save('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/heat_index',hit2)

In [ ]:
np.save('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/mask_shape',mask_shape)

In [ ]:
np.save('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/lat_obs_ncar',lat_obs.data)
np.save('/content/drive/MyDrive/BiasCorr/output_rh_tas_daily/lon_obs_ncar',lon_obs.data)

In [ ]:
N1=hit[0].shape[0]
N2=hit[0].shape[1]
print(N1,N2)
out_write=np.zeros(13,N1,N2)
for d in range(13):


# Other

In [ ]:
def map_plot_cyl_final(data_lat_x_lon,lat_cen,lon_cen,map_bounds,clab,y_limit,ax,mp_spacing=10,subplot_iter=1,loff=1,coff=1,ex='both'):

  lat_box=uu.cen_to_box(lat_cen)
  lon_box=uu.cen_to_box(lon_cen)

  map = Basemap(projection='cyl',resolution='l',
                llcrnrlon=map_bounds[0], 
                urcrnrlat=map_bounds[1],
                urcrnrlon=map_bounds[2], 
                llcrnrlat=map_bounds[3])


  xx, yy = meshgrid(lon_box,lat_box )
  #return  dimenion of lat * lon
  map.pcolor(xx, yy, data_lat_x_lon,cmap='jet')
  map.drawmeridians(np.arange(-180,180,mp_spacing),labels=[0,0,0,1], linewidth=1.0) #longitudes
  if loff==0:
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[1,0,0,0], linewidth=1.0) #latitudes
  else:
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[0,0,0,0], linewidth=1.0) #latitudes
  cbar = map.colorbar(extend=ex)
  #cbar.ax.set_yticklabels(['0','1','2','>3'])
  cbar.set_label(clab, rotation=90)
  plt.clim(y_limit)

  if coff==1:
    cbar.remove()

  #map.drawcountries(linewidth=1)
  map.readshapefile('/content/drive/MyDrive/Shapefiles/ind', 'IND', linewidth=1.0)

  map.drawcoastlines(linewidth=1)
  lxx=0.02
  lyy=0.98
  lcol='k'
  lsize=24
  lfamily='Tahoma'
  lweight=800
 # y_limit=[0,30]
  #clab='days'
  
  plt.text(lxx, lyy, chr(subplot_iter+96), horizontalalignment='left',verticalalignment='top', transform=ax.transAxes,fontweight=lweight,
           fontfamily=lfamily,fontsize=lsize,color=lcol)

In [ ]:
Dl=[0,2,6,10]

dapa=[R10mmindic[Dl[0],:,:],
  R10mmindic[Dl[1],:,:],
  R10mmindic[Dl[2],:,:],
  R10mmindic[Dl[3],:,:],
  CDDindic[Dl[0],:,:],
  CDDindic[Dl[1],:,:],
  CDDindic[Dl[2],:,:],
  CDDindic[Dl[3],:,:],
  Rx1dayindic[Dl[0],:,:],
  Rx1dayindic[Dl[1],:,:],
  Rx1dayindic[Dl[2],:,:],
  Rx1dayindic[Dl[3],:,:],
  R99pTOTindic[Dl[0],:,:],
  R99pTOTindic[Dl[1],:,:],
  R99pTOTindic[Dl[2],:,:],
  R99pTOTindic[Dl[3],:,:]]

clab='1111'
y_limits=[[0,120],
         [0,150],
         [0,120],
         [0,15]]
labels=['# of days','# of days','mm','%age']
lofff=[0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]
cofff=[1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0]
titlesx=['Observation','Raw CanESM2','BCSD CanESM2','BCSD Random']
titlesy=['R10mm','CDD','Rx1day','R99pTOT']
ex='max'
fig=plt.figure(figsize=(16,17))

for rr in range(4):
  for cc in range(4):
    iter=rr*4+cc
    ax=plt.subplot(4,4,iter+1)
    #plt.pcolor(ma.masked_array(dapa[rr*4+cc], mask=mask))
    map_plot_cyl_final(ma.masked_array(dapa[iter], mask=mask),lat_obs,lon_obs,map_bounds,labels[rr],y_limits[rr],ax,mp_spacing=10,subplot_iter=iter+1,loff=lofff[iter],coff=cofff[iter],ex=ex)
    plt.title(titlesx[cc])


fig.text(0.08, 0.8, titlesy[0], va='center', ha='center', rotation='vertical', fontsize=16)
fig.text(0.08, 0.6, titlesy[1], va='center', ha='center', rotation='vertical', fontsize=16)
fig.text(0.08, 0.4, titlesy[2], va='center', ha='center', rotation='vertical', fontsize=16)
fig.text(0.08, 0.21, titlesy[3], va='center', ha='center', rotation='vertical', fontsize=16)



In [ ]:
ind1h=get_corr_n_rmse(dapa[:4])

In [ ]:
ind2h=get_corr_n_rmse(dapa[4:8])

In [ ]:
ind3h=get_corr_n_rmse(dapa[8:12])

In [ ]:
ind4h=get_corr_n_rmse(dapa[12:16])

In [ ]:


def plot_hmap(data_corr,ylim,titles,fmt,cmap):
  fig=plt.figure(figsize=(25,13.75))
  S=data_corr[0].shape[0]
  for iii in range(4):
    for subs in range(S):
      ax=plt.subplot(4,6,iii*6+subs+1)
      
      data_set = data_corr[iii][subs,:,:]
      h=sns.heatmap(data_set , linewidth = 2 , cmap = cmap, 
                  annot=True,
                  fmt=fmt ,square=True,
                  xticklabels=titles,
                  yticklabels=titles,vmin=ylim[iii][0],vmax=ylim[iii][1])
      if iii==0:
        plt.title(region_shapes['State_Name'][subs])
        
      lxx=0.98
      lyy=0.98
      lcol='k'
      lsize=24
      lfamily='Tahoma'
      lweight=800
      
      plt.text(lxx, lyy,  chr(iii*4+subs+1+96), horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight=lweight,
              fontsize=lsize,color=lcol)    
      
  fig.text(0.08, 0.8, titlesy[0], va='center', ha='center', rotation='vertical', fontsize=16)
  fig.text(0.08, 0.6, titlesy[1], va='center', ha='center', rotation='vertical', fontsize=16)
  fig.text(0.08, 0.4, titlesy[2], va='center', ha='center', rotation='vertical', fontsize=16)
  fig.text(0.08, 0.21, titlesy[3], va='center', ha='center', rotation='vertical', fontsize=16)
  #fig.text(0.08, 0.21, titlesy[3], va='center', ha='center', rotation='vertical', fontsize=16)


  plt.show()

In [ ]:
titles=['Obs','GCM','BC-G','BC-R']


In [ ]:
plot_hmap([ind1h,ind2h,ind3h,ind4h],[[0,40],[0,50],[0,80],[0,10]],titles,".1f",'Reds')